In [ ]:
import os
from PIL import Image


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import math
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import roc_curve
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_examples = 19900
test_examples = 6774
validation_examples = 6451

img_height = img_width = 224
batch_size = 50


In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=15,
    zoom_range=(0.95, 0.95),
    horizontal_flip=True,
    vertical_flip=True,
    data_format="channels_last",
    dtype=None,
)

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1.0 / 255, dtype=None)
test_datagen = ImageDataGenerator(rescale=1.0 / 255, dtype=None)


In [ ]:
train_gen = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/data/train/",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode="binary",
    shuffle=True,
    seed=123,
)


Found 19900 images belonging to 2 classes.


In [ ]:
validation_gen = validation_datagen.flow_from_directory(
    "/content/drive/MyDrive/data/validation",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode="binary",
    shuffle=True,
    seed=123,
)


Found 6451 images belonging to 2 classes.


In [ ]:
test_gen = test_datagen.flow_from_directory(
    "/content/drive/MyDrive/data/test",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode="binary",
    shuffle=True,
    seed=123,
)

Found 6774 images belonging to 2 classes.


In [ ]:
METRICS = [
    keras.metrics.BinaryAccuracy(name="accuracy"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
    keras.metrics.AUC(name="auc"),
]

In [ ]:
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_v3/feature_vector/4",
                   trainable=True),  # Can be True, see below.
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=[keras.losses.BinaryCrossentropy(from_logits=False)],
    metrics=METRICS,
)


In [ ]:
history = model.fit(
    train_gen,
    epochs=5,
    verbose=2,
    steps_per_epoch=train_examples // batch_size,
    validation_data=validation_gen,
    validation_steps=validation_examples // batch_size,
    )


Epoch 1/5
398/398 - 17443s - loss: 0.3102 - accuracy: 0.9786 - precision: 0.0244 - recall: 0.0057 - auc: 0.7690 - val_loss: 0.2869 - val_accuracy: 0.9840 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.8320 - 17443s/epoch - 44s/step
Epoch 2/5
398/398 - 4194s - loss: 0.2853 - accuracy: 0.9824 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.8257 - val_loss: 0.2945 - val_accuracy: 0.9840 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6420 - 4194s/epoch - 11s/step
Epoch 3/5
398/398 - 4266s - loss: 0.2711 - accuracy: 0.9825 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.8117 - val_loss: 0.2587 - val_accuracy: 0.9840 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.8156 - 4266s/epoch - 11s/step
Epoch 4/5
398/398 - 4126s - loss: 0.2516 - accuracy: 0.9825 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.8219 - val_loss: 0.2696 - val_accuracy: 0.9840 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.5843 - 4

In [ ]:
def plot_roc(labels, data):
    predictions = model.predict(data)
    fp, tp, _ = roc_curve(labels, predictions)

    plt.plot(100 * fp, 100 * tp)
    plt.xlabel("False positives [%]")
    plt.ylabel("True positives [%]")
    plt.show()

In [ ]:
test_labels = np.array([])
num_batches = 0

for _, y in test_gen:
    test_labels = np.append(test_labels, y)
    num_batches += 1
    if num_batches == math.ceil(test_examples / batch_size):
        break

plot_roc(test_labels, test_gen)
model.evaluate(validation_gen, verbose=2)
model.evaluate(test_gen, verbose=2)

NameError: ignored

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['auc'])
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('auc')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc='upper left')

print(history.history.keys())

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


NameError: ignored